In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Kaggle linki: https://www.kaggle.com/code/brakurun/hearth-attack-logistic-regression

In [2]:
df = pd.read_csv("heart_attack_prediction_dataset.csv")
df.head()

,Patient ID,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,BMW7812,67,Male,208,158/88,72,0,0,1,0,...,6.615001,261404,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0
1,CZE1114,21,Male,389,165/93,98,1,1,1,1,...,4.963459,285768,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0
2,BNI9906,21,Female,324,174/99,72,1,0,0,0,...,9.463426,235282,28.176571,587,4,4,France,Europe,Northern Hemisphere,0
3,JLN3497,84,Male,383,163/100,73,1,1,1,0,...,7.648981,125640,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0
4,GFO8847,66,Male,318,91/88,93,1,1,1,1,...,1.514821,160555,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8763 entries, 0 to 8762
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Patient ID                       8763 non-null   object 
 1   Age                              8763 non-null   int64  
 2   Sex                              8763 non-null   object 
 3   Cholesterol                      8763 non-null   int64  
 4   Blood Pressure                   8763 non-null   object 
 5   Heart Rate                       8763 non-null   int64  
 6   Diabetes                         8763 non-null   int64  
 7   Family History                   8763 non-null   int64  
 8   Smoking                          8763 non-null   int64  
 9   Obesity                          8763 non-null   int64  
 10  Alcohol Consumption              8763 non-null   int64  
 11  Exercise Hours Per Week          8763 non-null   float64
 12  Diet                

In [4]:
df.isnull().sum()

Patient ID                         0
Age                                0
Sex                                0
Cholesterol                        0
Blood Pressure                     0
Heart Rate                         0
Diabetes                           0
Family History                     0
Smoking                            0
Obesity                            0
Alcohol Consumption                0
Exercise Hours Per Week            0
Diet                               0
Previous Heart Problems            0
Medication Use                     0
Stress Level                       0
Sedentary Hours Per Day            0
Income                             0
BMI                                0
Triglycerides                      0
Physical Activity Days Per Week    0
Sleep Hours Per Day                0
Country                            0
Continent                          0
Hemisphere                         0
Heart Attack Risk                  0
dtype: int64

In [5]:
df.drop("Patient ID", axis=1, inplace=True)
df.head()

,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,67,Male,208,158/88,72,0,0,1,0,0,...,6.615001,261404,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0
1,21,Male,389,165/93,98,1,1,1,1,1,...,4.963459,285768,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0
2,21,Female,324,174/99,72,1,0,0,0,0,...,9.463426,235282,28.176571,587,4,4,France,Europe,Northern Hemisphere,0
3,84,Male,383,163/100,73,1,1,1,0,1,...,7.648981,125640,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0
4,66,Male,318,91/88,93,1,1,1,1,0,...,1.514821,160555,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0


"Patient ID" sütunu modelimde kullanmayacağım için "Patient ID" sütunu veri çerçevesinden kaldırıyorum.

In [6]:
X = df.drop('Heart Attack Risk', axis=1)
y = df['Heart Attack Risk']


lojistik regresyon modelinizin kategorik (categorical) verileri doğrudan işleyemez. Kategorik verileri, sayısal verilere dönüştürmeniz gerekmektedir. Bu dönüşüm işlemi genellikle "One-Hot Encoding" veya "Label Encoding" gibi yöntemlerle yapılır.

"Label Encoding" kategorik verileri sayısal etiketlere dönüştüren bir yöntemdir, ancak bu yöntem lojistik regresyon gibi algoritmalar için pek uygun değildir. "One-Hot Encoding" ise her kategoriye ayrı bir sütun oluşturarak veriyi dönüştürür. Bu, kategorik verileri uygun bir şekilde sayısal verilere dönüştürmek için daha yaygın olarak kullanılan bir yöntemdir.

Önce veri setinizde hangi sütunların kategorik olduğunu belirledim ve ardından, bu kategorik sütunları "One-Hot Encoding" kullanarak dönüştürdüm.

In [7]:
from sklearn.model_selection import train_test_split

X = df[['Age', 'Sex', 'Cholesterol', 'Blood Pressure', 'Heart Rate', 'Diabetes', 'Family History', 'Smoking', 'Obesity', 
        'Alcohol Consumption', 'Exercise Hours Per Week', 'Diet', 'Previous Heart Problems', 'Medication Use', 'Stress Level',
        'Sedentary Hours Per Day', 'Income', 'BMI', 'Triglycerides', 'Physical Activity Days Per Week', 'Sleep Hours Per Day',
        'Country', 'Continent', 'Hemisphere']]
y = df['Heart Attack Risk']

# Kategorik sütunları One-Hot Encoding yapın
X = pd.get_dummies(X, columns=['Sex', 'Blood Pressure', 'Diet', 'Country', 'Continent', 'Hemisphere'], drop_first=True)

# Şimdi One-Hot Encoding yapılmış veriyi kullanarak modeli tekrar eğitin
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Eğitim veri seti boyutu:',X_train.shape)
print("Test veri seti boyutu:", X_test.shape)
print("Eğitim çıkış boyutu:", y_train.shape)
print("Test çıkış boyutu:", y_test.shape)


Eğitim veri seti boyutu: (7010, 3960)
Test veri seti boyutu: (1753, 3960)
Eğitim çıkış boyutu: (7010,)
Test çıkış boyutu: (1753,)


Modelimizi eğitmeden önce değerlerimizi standardize etmemiz faydalı olacaktır.

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(X_test)

In [9]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(x_train_scaled, y_train)

LogisticRegression()

Parametre ve bias değerlerimizi görelim.

In [10]:
print(model.coef_)
print(model.intercept_)

[[ 0.08515822  0.05963791 -0.02645323 ... -0.01428559  0.0704677
   0.00490383]]
[-1.89174239]


In [11]:
y_head = model.predict(x_test_scaled)

Modeli değerlendirmek için sık kullanılan birkaç metriği deneyerek başarı oranlarını gözlemleyelim:

Doğruluk (Accuracy): Doğruluk, modelinizin doğru sınıflandırma oranını gösterir. Doğru tahminlerin toplam verilere oranıdır.

In [12]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_head)
print("Accuracy : %",accuracy)

Accuracy : % 0.5693097547062179


Hassasiyet (Precision): Hassasiyet, pozitif olarak tahmin edilen örneklerin kaçının gerçekten pozitif olduğunu gösterir.

In [13]:
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_head)
print("Precision: %", precision)


Precision: % 0.3727454909819639


Duyarlılık (Recall): Duyarlılık, gerçek pozitif örneklerin ne kadarının doğru bir şekilde pozitif olarak tahmin edildiğini gösterir. 

In [14]:
from sklearn.metrics import recall_score
recall = recall_score(y_test, y_head)

print("Recall: %", recall)


Recall: % 0.2961783439490446


F1 Skoru: F1 skoru, hassasiyet ve duyarlılığın harmonik ortalamasıdır ve hem false positive hem de false negative hataları dikkate alır. 

In [15]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_head)

print("F1 Score: %", f1)


F1 Score: % 0.3300798580301686


Tüm metrikleri alt alta print fonksiyonu ile yazarak düzenli şekilde tüm oranları görebiliriz.

In [16]:
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)

Accuracy:  0.5693097547062179
Precision:  0.3727454909819639
Recall:  0.2961783439490446
F1 Score:  0.3300798580301686


Görüldüğü üzere en iyi oran Doğruluk (Accuracy) metriksinde çıktı. Ama yine de çok iyi bir oran olduğu söylenemez. Peki daha iyi bir oran bulabilir miyiz?

Grid Search, belirli bir hiperparametre aralığında farklı değerleri deneyerek en iyi hiperparametre kombinasyonunu bulmanıza yardımcı olur. Bu, modelinizi daha iyi performanslı hale getirmenize yardımcı olabilir. Manüel olarak hiperparametreleri denemek yerine, __Grid Search__ otomatik olarak tüm kombinasyonları dener ve en iyi sonucu bulur.

In [17]:
from sklearn.model_selection import GridSearchCV

#lojistik regresyon modelinin düzenleme katsayısı (C)  
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}


grid_cv = GridSearchCV(model, param_grid, scoring = "accuracy", n_jobs=-1, cv=3)

grid_cv.fit(X_train, y_train)

en_iyi_model = grid_cv.best_estimator_


print("Best Params", grid_cv.best_params_)
print("Best CV Score", grid_cv.best_score_)

Best Params {'C': 0.001}
Best CV Score 0.6417974176587476


Manuel hesaplamalarımızda en iyi sonuç olarak 0.56 değerini bulmuştuk ama Grid Search parametresiyle işlem yaptığımızda bu oranın 0.64 e yükselebildiğini gözlemledik.

Scikit-learn kütüphanesi, hiper parametre optimizasyonu için farklı teknikler sunar. Bu parametrelerden biri Grid Search iken bir diğeri ise Randomized Search'tür. Grid Search yerine Randomized Search de kullanabiliriz.

Ama bunun sonucunda da en iyi skorun aynı olduğu gözlemlenmekte.

In [18]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

#C hiperparametresi için 0.001 ile 100 arasında rastgele bir değer seçilmesi sağlanır
param_dist = {
    'C': uniform(loc=0.001, scale=100)
}

random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=10,
                                   scoring="accuracy", n_jobs=-1, cv=3)

random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_
best_params = random_search.best_params_

y_pred = best_model.predict(X_test)

print("Best Params", best_params)

print("Best CV Score", random_search.best_score_)

Best Params {'C': 47.936268660270024}
Best CV Score 0.6417974176587476


In [19]:
import joblib


In [20]:
#Modeli kaydetmek için joblib veya pickle gibi Python kütüphaneleri kullanılır.
#import pickle
import joblib

# Modeli diske kaydet
model_filename = "lojistik_regresyon_model.pkl"
joblib.dump(en_iyi_model, model_filename)


['lojistik_regresyon_model.pkl']

Bu kod, en_iyi_model adlı en iyi modeli "lojistik_regresyon_model.pkl" adlı bir dosyaya kaydeder. Ardından çağırma işlemi bu dosya üzerinden yapılır ve tüm kod çalıştırılmadan model kaydedilmiş olur.

In [21]:
#dosyadan modeli yükler. modelin yeni ismi de loaded_model olur

loaded_model = joblib.load("lojistik_regresyon_model.pkl")

In [22]:
y_pred = loaded_model.predict(X_test)

loaded_model adlı yüklenen modeli kullanarak X_test verileri üzerinde tahminler yapabiliriz. Tüm kodu çalıştırmak yerine sadece loaded_model i çağırmak yeterli olacaktır.